In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

input_path = 'images/'
output_path = 'output/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

if not os.path.exists(output_path + 'proj/' ):
    os.makedirs(output_path + 'proj/' )

if not os.path.exists(output_path + 'transf/' ):
    os.makedirs(output_path + 'transf/' )

# 1 - Medidas dos Objetos

In [7]:
# Carregar as imagens
imagens = []
for nome in sorted(os.listdir('images/')):
    if nome.endswith('.png'):
        imagem = cv2.imread(f'./images/{nome}', cv2.IMREAD_COLOR)
        imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        imagens.append((imagem, nome[:-4]))

libpng warning: iCCP: profile 'default_rgb.icc': 0h: PCS illuminant is not D50
libpng warning: iCCP: profile 'default_rgb.icc': 0h: PCS illuminant is not D50


## 1.1 - Transformação de Cores